In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import importlib

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets.hdf5 import get_train_loaders
from unet3d.config import load_config
from unet3d.losses import get_loss_criterion
from unet3d.metrics import get_evaluation_metric
from unet3d.model import get_model
from unet3d.trainer import UNet3DTrainer
from unet3d.utils import get_logger
from unet3d.utils import get_number_of_learnable_parameters

import argparse

import os
import yaml
import numpy as np

In [3]:
config_file = 'resources/train_config_dice.yaml'
config = yaml.load(open(config_file, 'r'))
model = get_model(config)
loaded_model = torch.load('3dunet/best_checkpoint.pytorch')
model.load_state_dict(loaded_model['model_state_dict'])

/home/gheimber/software/anaconda3/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [8]:
prediction = model(torch.zeros(1,1,50,50,50))
prediction_np = prediction.detach().numpy()

In [9]:
prediction_np

array([[[[[-0.7812276 , -0.9906611 , -1.0881246 , ..., -0.4890366 ,
           -0.34127688, -0.2731741 ],
          [-1.2083604 , -1.414762  , -1.4640585 , ..., -0.6194863 ,
           -0.4752579 , -0.34453893],
          [-1.2981142 , -1.5110765 , -1.5435966 , ..., -0.35375926,
           -0.24357384, -0.18988967],
          ...,
          [-0.76230234, -0.7427946 , -0.6649752 , ..., -0.034718  ,
            0.02723649,  0.09578961],
          [-0.45462725, -0.2446384 , -0.12134302, ...,  0.41373178,
            0.43876433,  0.33645323],
          [-0.4929939 , -0.33749488, -0.21530782, ...,  0.19455962,
            0.16958295,  0.17544071]],

         [[-1.1996089 , -1.5300776 , -1.7381389 , ..., -1.7668756 ,
           -1.5575361 , -1.0574166 ],
          [-1.8464499 , -1.9631349 , -2.0932267 , ..., -2.0983443 ,
           -2.009969  , -1.6647013 ],
          [-1.9409184 , -2.080535  , -2.204804  , ..., -2.0876842 ,
           -1.9608508 , -1.6128749 ],
          ...,
          [-1.

In [6]:
import ipyvolume as ipv
ipv.quickvolshow(np.squeeze(prediction_np))